In [1]:
# Read data from the same train_X and train_y used to get good results on tensorflow
import pickle

with open('Librispeech_Tensorflow_X_train.pkl','rb') as f:
    X_train = pickle.load(f)
with open('Librispeech_Tensorflow_y_train.pkl','rb') as f:
    y_train = pickle.load(f)
with open('Librispeech_Tensorflow_X_val.pkl','rb') as f:
    X_val = pickle.load(f)
with open('Librispeech_Tensorflow_y_val.pkl','rb') as f:
    y_val = pickle.load(f)
with open('Librispeech_Tensorflow_X_test.pkl','rb') as f:
    X_test = pickle.load(f)
with open('Librispeech_Tensorflow_y_test.pkl','rb') as f:
    y_test = pickle.load(f)

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, parameters, labels, device):
        self.parameters = parameters
        self.labels = labels
        self.device = device

    def __len__(self):
        return len(self.parameters)

    def __getitem__(self, idx):
        param = torch.tensor(self.parameters[idx], dtype=torch.float32).to(device)
        label = torch.tensor(self.labels[idx].argmax(), dtype=torch.long).to(device)
        return param, label

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device {device}")


Using device cpu


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

class OldNeuralNetwork(nn.Module):
    def __init__(self):
        super(OldNeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(193, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 251)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.25)
        self.dropout3 = nn.Dropout(0.5)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.dropout3(x)
        x = self.softmax(x)
        #print(x)
        return x

In [5]:
old_model = OldNeuralNetwork()
old_model.to(device)

OldNeuralNetwork(
  (fc1): Linear(in_features=193, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=251, bias=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.25, inplace=False)
  (dropout3): Dropout(p=0.5, inplace=False)
  (softmax): Softmax(dim=None)
)

In [6]:
from torchinfo import summary

summary(old_model, input_size=(1, 193))

C:\Users\achantrill\AppData\Local\anaconda3\envs\MachineLearning\lib\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Layer (type:depth-idx)                   Output Shape              Param #
OldNeuralNetwork                         [1, 251]                  --
├─Linear: 1-1                            [1, 128]                  24,832
├─Dropout: 1-2                           [1, 128]                  --
├─Linear: 1-3                            [1, 128]                  16,512
├─Dropout: 1-4                           [1, 128]                  --
├─Linear: 1-5                            [1, 251]                  32,379
├─Dropout: 1-6                           [1, 251]                  --
├─Softmax: 1-7                           [1, 251]                  --
Total params: 73,723
Trainable params: 73,723
Non-trainable params: 0
Total mult-adds (M): 0.07
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.29
Estimated Total Size (MB): 0.30

In [17]:
class Trainer:
    def __init__(self, model, train_loader, val_loader, criterion, optimizer):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.current_epoch = 0

    def train_one_epoch(self):
        # Train loop
        self.model.train()
        train_loss_accumulator = 0.0
        train_correct_accumulator = 0
        train_total_samples =0
        for batch_idx, (data, targets) in enumerate(self.train_loader):
            self.optimizer.zero_grad()
            outputs = self.model(data)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss_accumulator += loss.item() * data.size(0)
            _, predicted = torch.max(outputs, 1)
            train_correct_accumulator += (predicted==targets).sum().item()
            train_total_samples += data.size(0)
        train_loss = train_loss_accumulator / train_total_samples
        train_accuracy = train_correct_accumulator / train_total_samples

        #Validation loop
        self.model.eval()
        val_loss_accumulator = 0.0
        val_correct_accumulator = 0
        val_total_samples = 0
        with torch.no_grad():
            val_loss = 0
            for data, targets in self.val_loader:
                outputs = self.model(data)
                loss = criterion(outputs, targets)
                val_loss_accumulator += loss.item() * data.size(0)
                _, predicted = torch.max(outputs, 1)
                val_correct_accumulator += (predicted==targets).sum().item()
                val_total_samples += data.size(0)
            val_loss = val_loss_accumulator / val_total_samples
            val_accuracy = val_correct_accumulator / val_total_samples
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {(train_accuracy * 100):.2f}%")
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {(val_accuracy * 100):.2f}%")
        return val_loss

    def train(self, epochs, patience, optimizer=None):
        if optimizer:
            self.optimizer = optimizer
        best_val_loss = float('inf')
        counter = 0
        from_epoch = self.current_epoch
        to_epoch = self.current_epoch + epochs
        for epoch in range(from_epoch, to_epoch):
            self.current_epoch += 1
            print(f"Epoch {self.current_epoch} / {to_epoch}")
            val_loss = self.train_one_epoch()
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                counter = 0
            else:
                counter += 1
                if counter >= patience:
                    print("Early stopping triggered")
                    break


In [18]:
train_loader = torch.utils.data.DataLoader(CustomDataset(X_train, y_train, device), batch_size=256, shuffle=True)
val_loader = torch.utils.data.DataLoader(CustomDataset(X_val, y_val, device), batch_size=256)

criterion = nn.CrossEntropyLoss()
# criterion = nn.functional.cross_entropy
optimizer = optim.Adam(old_model.parameters(), lr=0.001)

trainer = Trainer(old_model, train_loader, val_loader, criterion, optimizer)
trainer.train(100, 10)


Epoch 1 / 100
Train Loss: 5.4561, Train Accuracy: 9.22%
Validation Loss: 5.4066, Validation Accuracy: 26.21%
Epoch 2 / 100
Train Loss: 5.3732, Train Accuracy: 18.33%
Validation Loss: 5.2804, Validation Accuracy: 48.12%
Epoch 3 / 100
Train Loss: 5.2986, Train Accuracy: 25.71%
Validation Loss: 5.1512, Validation Accuracy: 60.02%
Epoch 4 / 100
Train Loss: 5.2495, Train Accuracy: 30.18%
Validation Loss: 5.0637, Validation Accuracy: 66.33%
Epoch 5 / 100
Train Loss: 5.2188, Train Accuracy: 32.84%
Validation Loss: 5.0004, Validation Accuracy: 69.77%
Epoch 6 / 100
Train Loss: 5.1976, Train Accuracy: 34.83%
Validation Loss: 4.9490, Validation Accuracy: 71.85%
Epoch 7 / 100
Train Loss: 5.1874, Train Accuracy: 35.58%
Validation Loss: 4.9227, Validation Accuracy: 72.79%
Epoch 8 / 100
Train Loss: 5.1780, Train Accuracy: 36.36%
Validation Loss: 4.8912, Validation Accuracy: 74.69%
Epoch 9 / 100
Train Loss: 5.1667, Train Accuracy: 37.36%
Validation Loss: 4.8612, Validation Accuracy: 76.48%
Epoch 10 / 

In [19]:
# Continue training
trainer.train(100, 50)

Epoch 101 / 200
Train Loss: 5.0701, Train Accuracy: 46.16%
Validation Loss: 4.6154, Validation Accuracy: 92.11%
Epoch 102 / 200
Train Loss: 5.0606, Train Accuracy: 47.05%
Validation Loss: 4.6117, Validation Accuracy: 92.47%
Epoch 103 / 200
Train Loss: 5.0598, Train Accuracy: 47.18%
Validation Loss: 4.6120, Validation Accuracy: 92.43%
Epoch 104 / 200
Train Loss: 5.0657, Train Accuracy: 46.65%
Validation Loss: 4.6116, Validation Accuracy: 92.47%
Epoch 105 / 200
Train Loss: 5.0652, Train Accuracy: 46.59%
Validation Loss: 4.6114, Validation Accuracy: 92.54%
Epoch 106 / 200
Train Loss: 5.0632, Train Accuracy: 46.81%
Validation Loss: 4.6123, Validation Accuracy: 92.26%
Epoch 107 / 200
Train Loss: 5.0648, Train Accuracy: 46.67%
Validation Loss: 4.6094, Validation Accuracy: 92.65%
Epoch 108 / 200
Train Loss: 5.0577, Train Accuracy: 47.38%
Validation Loss: 4.6102, Validation Accuracy: 92.54%
Epoch 109 / 200
Train Loss: 5.0727, Train Accuracy: 45.91%
Validation Loss: 4.6125, Validation Accuracy:

In [21]:
# Adjust learning rate and continue training
optimizer = optim.Adam(old_model.parameters(), lr=0.0001)
trainer.train(100, 50)

Epoch 301 / 400
Train Loss: 5.0560, Train Accuracy: 47.53%
Validation Loss: 4.5905, Validation Accuracy: 94.16%
Epoch 302 / 400
Train Loss: 5.0533, Train Accuracy: 47.80%
Validation Loss: 4.5904, Validation Accuracy: 94.19%
Epoch 303 / 400
Train Loss: 5.0601, Train Accuracy: 47.18%
Validation Loss: 4.5902, Validation Accuracy: 94.23%
Epoch 304 / 400
Train Loss: 5.0533, Train Accuracy: 47.79%
Validation Loss: 4.5901, Validation Accuracy: 94.23%
Epoch 305 / 400
Train Loss: 5.0532, Train Accuracy: 47.86%
Validation Loss: 4.5901, Validation Accuracy: 94.23%
Epoch 306 / 400
Train Loss: 5.0618, Train Accuracy: 46.96%
Validation Loss: 4.5901, Validation Accuracy: 94.26%
Epoch 307 / 400
Train Loss: 5.0605, Train Accuracy: 47.13%
Validation Loss: 4.5902, Validation Accuracy: 94.19%
Epoch 308 / 400
Train Loss: 5.0511, Train Accuracy: 48.00%
Validation Loss: 4.5901, Validation Accuracy: 94.19%
Epoch 309 / 400
Train Loss: 5.0559, Train Accuracy: 47.53%
Validation Loss: 4.5900, Validation Accuracy:

In [22]:
# Use test data to test
import random

samples = random.sample(range(X_test.shape[0]),100)

total_correct = 0
old_model.eval()

for i in range(len(samples)):
    truth = y_test.argmax(1)[samples[i]]
    prediction = old_model(torch.tensor([X_test[samples[i]]], dtype=torch.float32)).argmax()
    if truth == prediction:
        total_correct += 1
    # print(f"{i}: {truth} - {prediction}")
print(f"{total_correct}/{len(samples)} = {total_correct * 100 / len(samples):.2f}%")

93/100 = 93.00%


C:\Users\achantrill\AppData\Local\Temp\ipykernel_13412\2597662702.py:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:278.)
  prediction = old_model(torch.tensor([X_test[samples[i]]], dtype=torch.float32)).argmax()


In [24]:
# Let's try with a model that is slightly closer to what Jurgen was using (the previous model was missing a layer)
# This model has four fully connected layers with a relu function between each layer.
import torch
import torch.nn as nn
import torch.optim as optim

class NewNeuralNetwork(nn.Module):
    def __init__(self):
        super(NewNeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(193, 193)
        self.fc2 = nn.Linear(193, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 251)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.25)
        self.dropout3 = nn.Dropout(0.5)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.dropout2(x)
        x = torch.nn.functional.relu(self.fc3(x))
        x = self.dropout3(x)
        x = self.fc4(x)
        x = self.softmax(x)
        #print(x)
        return x

In [25]:
# Train the updated model:
new_model = NewNeuralNetwork()
new_model.to(device)

train_loader = torch.utils.data.DataLoader(CustomDataset(X_train, y_train, device), batch_size=256, shuffle=True)
val_loader = torch.utils.data.DataLoader(CustomDataset(X_val, y_val, device), batch_size=256)

criterion = nn.CrossEntropyLoss()
# criterion = nn.functional.cross_entropy
optimizer = optim.Adam(new_model.parameters(), lr=0.001)

new_trainer = Trainer(new_model, train_loader, val_loader, criterion, optimizer)

In [26]:
new_trainer.train(100, 10)

Epoch 1 / 100
Train Loss: 5.5161, Train Accuracy: 1.97%
Validation Loss: 5.4887, Validation Accuracy: 4.45%
Epoch 2 / 100
Train Loss: 5.4673, Train Accuracy: 6.93%
Validation Loss: 5.4041, Validation Accuracy: 14.13%
Epoch 3 / 100
Train Loss: 5.3955, Train Accuracy: 15.10%
Validation Loss: 5.2963, Validation Accuracy: 25.53%
Epoch 4 / 100
Train Loss: 5.3247, Train Accuracy: 22.37%
Validation Loss: 5.2116, Validation Accuracy: 33.88%
Epoch 5 / 100
Train Loss: 5.2611, Train Accuracy: 28.74%
Validation Loss: 5.1428, Validation Accuracy: 40.16%
Epoch 6 / 100
Train Loss: 5.2109, Train Accuracy: 33.84%
Validation Loss: 5.0950, Validation Accuracy: 44.64%
Epoch 7 / 100
Train Loss: 5.1606, Train Accuracy: 38.86%
Validation Loss: 5.0587, Validation Accuracy: 48.30%
Epoch 8 / 100
Train Loss: 5.1263, Train Accuracy: 42.13%
Validation Loss: 5.0198, Validation Accuracy: 52.24%
Epoch 9 / 100
Train Loss: 5.0939, Train Accuracy: 45.29%
Validation Loss: 5.0023, Validation Accuracy: 53.60%
Epoch 10 / 10

In [27]:
# Continue Training
new_trainer.train(100, 10)

Epoch 101 / 200
Train Loss: 4.8210, Train Accuracy: 71.28%
Validation Loss: 4.8101, Validation Accuracy: 72.18%
Epoch 102 / 200
Train Loss: 4.8204, Train Accuracy: 71.35%
Validation Loss: 4.8085, Validation Accuracy: 72.32%
Epoch 103 / 200
Train Loss: 4.8182, Train Accuracy: 71.59%
Validation Loss: 4.8085, Validation Accuracy: 72.36%
Epoch 104 / 200
Train Loss: 4.8176, Train Accuracy: 71.63%
Validation Loss: 4.8077, Validation Accuracy: 72.43%
Epoch 105 / 200
Train Loss: 4.8191, Train Accuracy: 71.49%
Validation Loss: 4.8056, Validation Accuracy: 72.68%
Epoch 106 / 200
Train Loss: 4.8178, Train Accuracy: 71.61%
Validation Loss: 4.8065, Validation Accuracy: 72.61%
Epoch 107 / 200
Train Loss: 4.8166, Train Accuracy: 71.79%
Validation Loss: 4.8061, Validation Accuracy: 72.64%
Epoch 108 / 200
Train Loss: 4.8163, Train Accuracy: 71.75%
Validation Loss: 4.8038, Validation Accuracy: 72.89%
Epoch 109 / 200
Train Loss: 4.8150, Train Accuracy: 71.88%
Validation Loss: 4.8004, Validation Accuracy:

In [28]:
# Adjust learning rate and continue training
optimizer = optim.Adam(new_model.parameters(), lr=0.0001)
trainer.train(100, 50)

Epoch 383 / 482
Train Loss: 5.0539, Train Accuracy: 47.68%
Validation Loss: 4.5898, Validation Accuracy: 94.23%
Epoch 384 / 482
Train Loss: 5.0549, Train Accuracy: 47.58%
Validation Loss: 4.5898, Validation Accuracy: 94.23%
Epoch 385 / 482
Train Loss: 5.0545, Train Accuracy: 47.67%
Validation Loss: 4.5898, Validation Accuracy: 94.23%
Epoch 386 / 482
Train Loss: 5.0563, Train Accuracy: 47.43%
Validation Loss: 4.5898, Validation Accuracy: 94.23%
Epoch 387 / 482
Train Loss: 5.0540, Train Accuracy: 47.69%
Validation Loss: 4.5898, Validation Accuracy: 94.23%
Epoch 388 / 482
Train Loss: 5.0542, Train Accuracy: 47.72%
Validation Loss: 4.5898, Validation Accuracy: 94.23%
Epoch 389 / 482
Train Loss: 5.0507, Train Accuracy: 48.05%
Validation Loss: 4.5898, Validation Accuracy: 94.23%
Epoch 390 / 482
Train Loss: 5.0518, Train Accuracy: 47.96%
Validation Loss: 4.5898, Validation Accuracy: 94.23%
Epoch 391 / 482
Train Loss: 5.0514, Train Accuracy: 48.06%
Validation Loss: 4.5898, Validation Accuracy:

In [29]:
# Adjust learning rate and continue training
optimizer = optim.Adam(new_model.parameters(), lr=0.00001)
trainer.train(100, 25)

Epoch 434 / 533
Train Loss: 5.0495, Train Accuracy: 48.28%
Validation Loss: 4.5898, Validation Accuracy: 94.23%
Epoch 435 / 533
Train Loss: 5.0612, Train Accuracy: 46.98%
Validation Loss: 4.5898, Validation Accuracy: 94.23%
Epoch 436 / 533
Train Loss: 5.0563, Train Accuracy: 47.49%
Validation Loss: 4.5898, Validation Accuracy: 94.23%
Epoch 437 / 533
Train Loss: 5.0506, Train Accuracy: 48.12%
Validation Loss: 4.5898, Validation Accuracy: 94.23%
Epoch 438 / 533
Train Loss: 5.0537, Train Accuracy: 47.84%
Validation Loss: 4.5898, Validation Accuracy: 94.23%
Epoch 439 / 533
Train Loss: 5.0512, Train Accuracy: 47.96%
Validation Loss: 4.5898, Validation Accuracy: 94.23%
Epoch 440 / 533
Train Loss: 5.0558, Train Accuracy: 47.46%
Validation Loss: 4.5898, Validation Accuracy: 94.23%
Epoch 441 / 533
Train Loss: 5.0551, Train Accuracy: 47.57%
Validation Loss: 4.5898, Validation Accuracy: 94.23%
Epoch 442 / 533
Train Loss: 5.0546, Train Accuracy: 47.63%
Validation Loss: 4.5898, Validation Accuracy: